# Construyendo una aplicación simple de LLM

En esta guía de inicio rápido, le mostraremos cómo crear una aplicación LLM sencilla con LangChain. Esta aplicación traducirá texto del inglés a otro idioma. Es una aplicación LLM relativamente sencilla: solo requiere una llamada LLM y algunas indicaciones. Aun así, es una excelente manera de comenzar con LangChain: ¡se pueden crear muchas funciones con solo algunas indicaciones y una llamada LLM!

Después de leer este tutorial, tendrá una visión general de:

- Uso de modelos de lenguaje

- Uso de plantillas de indicaciones


## Bibliotecas
- langchain
- langchain_ollama
- langchain_core
```bash
pip install langchain langchain_ollama langchain_core
```

In [3]:
from langchain_ollama import ChatOllama 
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Usando Modelos Largos de Lenguaje Local

In [4]:
# Model
model = ChatOllama(
    model="llama3.2",                  # Modelo descargado desde Ollama (puedes cambiar el nombre según el modelo disponible)
    base_url="http://localhost:11434"  # URL local donde corre el servidor de Ollama
)

## Usando el LLM
Primero, usemos el modelo directamente. Los ChatModels son instancias de los Runnables de LangChain, lo que significa que exponen una interfaz estándar para interactuar con ellos. Para llamar al modelo, podemos pasar una lista de mensajes al método .invoke.

In [9]:
# Using LLM
messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

# Using dict of messages
model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-09T23:43:51.63234Z', 'done': True, 'done_reason': 'stop', 'total_duration': 465263200, 'load_duration': 37064700, 'prompt_eval_count': 34, 'prompt_eval_duration': 103622700, 'eval_count': 4, 'eval_duration': 323202800, 'model_name': 'llama3.2'}, id='run-b6f94484-41b4-4cde-8883-17cce05d375b-0', usage_metadata={'input_tokens': 34, 'output_tokens': 4, 'total_tokens': 38})

Tenga en cuenta que los ChatModels reciben objetos de mensaje como entrada y los generan como salida. Además del contenido textual, los objetos de mensaje transmiten roles conversacionales y contienen datos importantes, como llamadas a herramientas y recuentos de uso de tokens.

LangChain también admite entradas de modelos de chat mediante cadenas o formato OpenAI. Los siguientes son equivalentes:

In [12]:
# Using direct string
model.invoke("Hello")

# Using dict with role and content
model.invoke([{"role": "user", "content": "Hello"}])

# Using Humanmessage with the prompt
model.invoke([HumanMessage("Hello")])

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-09T23:49:16.623069Z', 'done': True, 'done_reason': 'stop', 'total_duration': 796749600, 'load_duration': 28527600, 'prompt_eval_count': 26, 'prompt_eval_duration': 88446200, 'eval_count': 8, 'eval_duration': 679261700, 'model_name': 'llama3.2'}, id='run-58e56158-5cd7-41dd-888d-fe5c353c627c-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

## Streaming
Dado que los modelos de chat son ejecutables, exponen una interfaz estándar que incluye modos de invocación asíncronos y de transmisión. Esto nos permite transmitir tokens individuales desde un modelo de chat:

In [13]:
#Streaming
for token in model.stream(messages):
    print(token.content, end="|")

C|iao|!||

## Prompt Templates
Ahora mismo estamos pasando una lista de mensajes directamente al modelo de lenguaje. ¿De dónde proviene esta lista? Normalmente, se construye combinando la entrada del usuario y la lógica de la aplicación. Esta lógica de la aplicación suele tomar la entrada del usuario sin procesar y transformarla en una lista de mensajes listos para pasar al modelo de lenguaje. Las transformaciones comunes incluyen añadir un mensaje del sistema o formatear una plantilla con la entrada del usuario.

Las plantillas de solicitud son un concepto de LangChain diseñado para facilitar esta transformación. Reciben la entrada del usuario sin procesar y devuelven datos (una solicitud) listos para pasar a un modelo de lenguaje.

### String PromptTemplates
Estas plantillas de solicitud se utilizan para dar formato a una sola cadena y, generalmente, se usan para entradas más sencillas.
Por ejemplo, una forma común de construir y usar una plantilla de solicitud es la siguiente:

In [17]:
prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

# Assing topic value with invoke
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

text='Tell me a joke about cats'


### ChatPromptTemplates
Estas plantillas de mensajes se utilizan para dar formato a una lista de mensajes.
Estas "plantillas" consisten en una lista de plantillas.
Por ejemplo, una forma común de crear y usar una ChatPromptTemplate es la siguiente:

In [18]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder
Esta plantilla de mensaje se encarga de añadir una lista de mensajes en un lugar específico.
En la plantilla ChatPromptTemplate anterior, vimos cómo formatear dos mensajes, cada uno como una cadena.
¿Pero qué sucedería si quisiéramos que el usuario pasara una lista de mensajes que colocaríamos en un lugar específico?
Así es como se usa MessagesPlaceholder.

In [19]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
print(prompt)

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


Esto generará una lista de dos mensajes: el primero es un mensaje del sistema y el segundo es el HumanMessage que pasamos.
Si hubiéramos pasado 5 mensajes, se habrían generado 6 mensajes en total (el mensaje del sistema más los 5 pasados).
Esto es útil para permitir que una lista de mensajes se ubique en un lugar específico.

Una forma alternativa de lograr lo mismo sin usar la clase MessagesPlaceholder explícitamente es:

In [20]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
])

prompt = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
print(prompt)

messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


Creemos una plantilla de solicitud aquí. Aceptará dos variables de usuario:

- idioma: el idioma al que se traducirá el texto
- texto: el texto a traducir

In [21]:
# Prompt Template
system_template = "Translate the following from English into {language}"

# Prompt template with dict of messages using from_messages method
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Tenga en cuenta que ChatPromptTemplate admite varios roles de mensaje en una sola plantilla. Formateamos el parámetro de idioma en el mensaje del sistema y el texto del usuario en un mensaje de usuario.

La entrada de esta plantilla de mensaje es un diccionario. Podemos experimentar con esta plantilla de mensaje por sí sola para ver qué hace.

In [22]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})
print(prompt)

messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


Podemos ver que devuelve un ChatPromptValue que consta de dos mensajes. Si queremos acceder a los mensajes directamente, hacemos lo siguiente:

In [23]:
print(prompt.to_messages())

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


Finalmente, podemos invocar el modelo de chat en el mensaje formateado:

In [24]:
response = model.invoke(prompt)
print(response.content)

Ciao!
